In [ ]:
import os
import joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

# TensorFlow/Keras is required for the CNN model
try:
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    TENSORFLOW_AVAILABLE = True
except ImportError:
    print("Warning: TensorFlow/Keras not found. CNN training will be skipped.")
    TENSORFLOW_AVAILABLE = False


# --- Configuration and File Paths ---
RF_MODEL_PATH = "rf_pipeline.pkl"
CNN_MODEL_PATH = "cnn_model.h5"
FEATURE_DATASET_PATH = "fatigue_features.csv" # CSV file with all engineered features
IMAGE_DATA_DIR = "cnn_dataset/" # Directory structure for CNN images (e.g., cnn_dataset/open, cnn_dataset/closed)

# --- 1. Random Forest Training (Feature-Based Classification) ---

def train_random_forest(data_path):
    """
    Trains a Random Forest classifier on engineered features.

    Data should include: EAR, MAR, Pitch, Yaw, Roll, Brightness, Label
    """
    if not os.path.exists(data_path):
        print(f"Error: Feature dataset not found at {data_path}")
        print("Please ensure you have generated this CSV from your video processing.")
        return

    print("--- Starting Random Forest Training ---")
    df = pd.read_csv(data_path)

    # 1. Prepare Data (Features + Target)
    feature_cols = ['EAR', 'MAR', 'Pitch', 'Yaw', 'Roll', 'Brightness']
    target_col = 'Label'

    X = df[feature_cols].values
    y = df[target_col].values

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # 2. Define the Pipeline (StandardScaler for preprocessing + RF model)
    rf_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'))
    ])

    # 3. Train the Model
    rf_pipeline.fit(X_train, y_train)

    # 4. Evaluate and Save
    y_pred = rf_pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    print(f"\nRandom Forest Test Accuracy: {accuracy:.4f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    # Save the entire pipeline (scaler and model)
    joblib.dump(rf_pipeline, RF_MODEL_PATH)
    print(f"\n✅ Random Forest Pipeline saved successfully as {RF_MODEL_PATH}")


# --- 2. CNN Training (Image-Based Classification, e.g., Eye State) ---

def create_cnn_model(input_shape):
    """Defines a simple CNN architecture for eye/mouth classification."""
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # Binary classification (e.g., Open/Closed)
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

def train_cnn(data_dir):
    """Trains the CNN model using image data generators."""
    if not TENSORFLOW_AVAILABLE:
        return

    if not os.path.exists(data_dir):
        print(f"Error: CNN image data directory not found at {data_dir}")
        print("Please ensure you have structured your images (e.g., cnn_dataset/0 and cnn_dataset/1).")
        return

    print("\n--- Starting CNN Image Model Training ---")

    # Image parameters
    IMG_HEIGHT, IMG_WIDTH = 48, 48
    BATCH_SIZE = 32
    EPOCHS = 10

    # Data Augmentation and Preprocessing
    datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        validation_split=0.2
    )

    # Load training and validation data
    train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='binary',
        subset='training'
    )

    validation_generator = datagen.flow_from_directory(
        data_dir,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='binary',
        subset='validation'
    )

    # Build and Train Model
    input_shape = (IMG_HEIGHT, IMG_WIDTH, 3)
    cnn_model = create_cnn_model(input_shape)

    history = cnn_model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // BATCH_SIZE
    )

    # Save the Model
    cnn_model.save(CNN_MODEL_PATH)
    print(f"\n✅ CNN Model saved successfully as {CNN_MODEL_PATH}")


# --- Main Execution ---

if __name__ == "__main__":
    # Ensure the required CSV for feature model exists
    # Create a dummy CSV for testing the script structure if the real one isn't ready
    if not os.path.exists(FEATURE_DATASET_PATH):
        print(f"Creating a DUMMY dataset for {FEATURE_DATASET_PATH}...")
        dummy_data = {
            'EAR': np.random.uniform(0.15, 0.40, 1000),
            'MAR': np.random.uniform(0.10, 0.70, 1000),
            'Pitch': np.random.uniform(-30, 30, 1000),
            'Yaw': np.random.uniform(-30, 30, 1000),
            'Roll': np.random.uniform(-20, 20, 1000),
            'Brightness': np.random.uniform(0.3, 1.0, 1000),
            'Label': np.random.randint(0, 2, 1000)
        }
        df_dummy = pd.DataFrame(dummy_data)
        # Adjust labels to simulate fatigue: low EAR or high MAR/Pitch is fatigue (Label=1)
        df_dummy.loc[(df_dummy['EAR'] < 0.20) | (df_dummy['MAR'] > 0.50) | (abs(df_dummy['Pitch']) > 20), 'Label'] = 1
        df_dummy.to_csv(FEATURE_DATASET_PATH, index=False)
        print("NOTE: Training will run on DUMMY DATA. Replace with real data for production models.")

    # 1. Train the Random Forest (RF) model
    train_random_forest(FEATURE_DATASET_PATH)

    # 2. Train the CNN model (if TensorFlow is available)
    if TENSORFLOW_AVAILABLE:
        # NOTE: You must prepare your image data in the IMAGE_DATA_DIR folder
        # (e.g., cnn_dataset/0 for 'alert' images, cnn_dataset/1 for 'fatigue' images)
        # If the directory doesn't exist, this step will be skipped with a warning.
        train_cnn(IMAGE_DATA_DIR)

    print("\nTraining process complete. The generated models are ready for API integration.")

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import joblib
import os
import time

# --- Configuration ---
# ⚠️ Make sure this path points to the video file you uploaded or a file accessible in your environment.
VIDEO_PATH = "/content/drive/MyDrive/data_set/videoblocks-mah08252_24_bihvfqecc__c99dce9dd347502d4c4b714390d0a37d__P360.mp4"
MODEL_PATH = "/content/drive/MyDrive/data_set/eye_blinks_model.pkl"

# --- Mediapipe Setup ---
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Eye Landmark Indices for EAR calculation
EYE_INDICES = [33, 160, 158, 133, 153, 144]
# Fixed interpretation for prediction (adjust if prediction is inverted)
LABEL_NAMES = {0: "Closed", 1: "Open"}

# --- Feature Extraction Functions (Unchanged) ---

def aspect_ratio(landmarks, indices):
    """Calculates the Aspect Ratio (EAR) for the eye."""
    p1, p2, p3, p4, p5, p6 = [np.array(landmarks[i]) for i in indices]
    vertical1 = np.linalg.norm(p2 - p6)
    vertical2 = np.linalg.norm(p3 - p5)
    horizontal = np.linalg.norm(p1 - p4)
    ear = (vertical1 + vertical2) / (2.0 * horizontal + 1e-6)
    return ear

def extract_eye_feature(image):
    """Processes image to find face landmarks and calculate EAR."""
    h, w = image.shape[:2]
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_image)

    if not results.multi_face_landmarks:
        return None, None

    landmarks = [(lm.x * w, lm.y * h) for lm in results.multi_face_landmarks[0].landmark]
    ear = aspect_ratio(landmarks, EYE_INDICES)
    eye_points = [landmarks[i] for i in EYE_INDICES]

    return np.array([[ear]]), eye_points

# --- Main Prediction Function (Modified for Colab) ---

def run_eye_blink_detection(video_path, model_path):
    # 1. Load the Model
    try:
        if not os.path.exists(model_path):
             print(f"❌ ERROR: Model file not found at {model_path}. Please ensure it is uploaded.")
             return

        eye_model = joblib.load(model_path)
        print(f"✅ Successfully loaded Eye Blink Model from {model_path}.")
    except Exception as e:
        print(f"❌ ERROR loading model: {e}")
        return

    # 2. Setup Video Capture and Output Writer
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ ERROR: Could not open video file at {video_path}")
        return

    # Get video properties for output
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Define the output file name and video writer
    output_filename = "output_eye_detection.mp4"
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Codec for MP4
    out = cv2.VideoWriter(output_filename, fourcc, fps, (frame_width, frame_height))

    print(f"🎥 Starting prediction on video. Output saved to: {output_filename}")

    frame_count = 0
    is_closed = False
    blink_count = 0

    # Process video frames
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # 3. Extract Feature
        ear_feature, eye_points = extract_eye_feature(frame)

        prediction_text = "No Face Detected"

        if ear_feature is not None:
            # 4. Predict
            prediction = eye_model.predict(ear_feature)[0]

            # 5. Interpret and Update Blink Count
            current_state = LABEL_NAMES.get(prediction, "Unknown")
            prediction_text = f"Eye: {current_state}"

            if current_state == "Closed":
                if not is_closed:
                    is_closed = True
            elif current_state == "Open" and is_closed:
                blink_count += 1
                is_closed = False

            # 6. Visualization
            if eye_points:
                eye_points_np = np.array(eye_points, dtype=np.int32).reshape((-1, 1, 2))
                cv2.polylines(frame, [eye_points_np], isClosed=True, color=(0, 255, 0), thickness=2)

            cv2.putText(frame, f"EAR: {ear_feature[0][0]:.2f}", (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)

        # Display Prediction Text and Blink Count
        cv2.putText(frame, prediction_text, (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
        cv2.putText(frame, f"Blinks: {blink_count}", (frame.shape[1] - 180, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

        # 7. Write Frame to Output Video
        out.write(frame)

        # 8. Display one frame in the Colab output every 10 frames (optional, for monitoring)
        # We don't use cv2.imshow() here; we use cv2_imshow from google.colab.patches
        if frame_count % 10 == 0:
            # You can display a small frame for monitoring, but for a full video, it's best to save it.
            pass

    # 9. Cleanup
    cap.release()
    out.release()
    print("-" * 50)
    print(f"✅ Processing complete. Total Frames Processed: {frame_count}")
    print(f"🎬 Output video saved as {output_filename}. You can now download it.")
    print(f"Total Blinks Detected: {blink_count}")
    print("-" * 50)


# --- Execute ---
# Run this function to start the video processing
run_eye_blink_detection(VIDEO_PATH, MODEL_PATH)

✅ Successfully loaded Eye Blink Model from /content/drive/MyDrive/data_set/eye_blinks_model.pkl.
🎥 Starting prediction on video. Output saved to: output_eye_detection.mp4
--------------------------------------------------
✅ Processing complete. Total Frames Processed: 532
🎬 Output video saved as output_eye_detection.mp4. You can now download it.
Total Blinks Detected: 0
--------------------------------------------------


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib
import zipfile
import shutil

# ----- Step 1: Extract ZIPs (Using corrected paths) -----
yawn_zip = r"/content/drive/MyDrive/data_set/archive (4).zip"
eye_zip = r"/content/drive/MyDrive/data_set/test_eye_data.zip.zip"

# Corrected accessible paths for Colab/Linux
extract_yawn = r"/content/yawn_dataset"
extract_eye = r"/content/eye_blink_dataset"

def safe_extract(zip_path, extract_path):
    if os.path.exists(extract_path):
        print(f"Directory already exists: {extract_path}")
        return

    if not os.path.exists(os.path.dirname(extract_path)):
        os.makedirs(os.path.dirname(extract_path), exist_ok=True)

    print(f"Extracting {zip_path} to {extract_path}...")
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
        print(f"Extraction successful: {extract_path}")
    except FileNotFoundError:
        print(f"❌ ERROR: Zip file not found at {zip_path}. Check your Drive path!")
    except Exception as e:
        print(f"❌ ERROR during extraction: {e}")

safe_extract(yawn_zip, extract_yawn)
safe_extract(eye_zip, extract_eye)

print("✅ Datasets extraction process complete.\n")

# ----- Step 2: Mediapipe setup -----
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True)

def aspect_ratio(landmarks, indices):
    """Calculates the Aspect Ratio (similar to EAR/MAR)."""
    p1, p2, p3, p4, p5, p6 = [np.array(landmarks[i]) for i in indices]
    vertical1 = np.linalg.norm(p2 - p6)
    vertical2 = np.linalg.norm(p3 - p5)
    horizontal = np.linalg.norm(p1 - p4)
    # Added a small epsilon to avoid division by zero, although highly unlikely
    return (vertical1 + vertical2) / (2.0 * horizontal + 1e-6)

def extract_features(image, part="eye"):
    h, w = image.shape[:2]
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if not results.multi_face_landmarks:
        return None
    landmarks = [(lm.x * w, lm.y * h) for lm in results.multi_face_landmarks[0].landmark]

    if part == "eye":
        # Indices for the right eye
        EYE = [33, 160, 158, 133, 153, 144]
        feature = aspect_ratio(landmarks, EYE)
    else:
        # Indices for the mouth
        MOUTH = [78, 308, 13, 14, 87, 317]
        feature = aspect_ratio(landmarks, MOUTH)
    return [feature]

def load_dataset(path, part="eye"):
    """Loads images with explicit label mapping to prevent inversion and performs data integrity check."""
    X, y = [], []

    # Explicitly define the labels
    if part == "eye":
        # We want closed eyes (low EAR) to be label 0, open eyes (high EAR) to be label 1.
        LABEL_MAP = {"closed": 0, "open": 1, "Closed": 0, "Open": 1}
        print(f"  -> Eye labels: Closed={LABEL_MAP['Closed']}, Open={LABEL_MAP['Open']}")
    elif part == "mouth":
        # We want non-yawn (low MAR) to be label 0, yawn (high MAR) to be label 1.
        LABEL_MAP = {"no_yawn": 0, "yawn": 1, "No_Yawn": 0, "Yawn": 1}
        print(f"  -> Yawn labels: No Yawn={LABEL_MAP['no_yawn']}, Yawn={LABEL_MAP['yawn']}")
    else:
        return np.array(X), np.array(y)

    for folder_name in os.listdir(path):
        folder_path = os.path.join(path, folder_name)

        # Get label from map (handling case-insensitivity)
        label = LABEL_MAP.get(folder_name)
        if label is None:
            label = LABEL_MAP.get(folder_name.lower())

        if label is not None and os.path.isdir(folder_path):
            print(f"  -> Processing folder: {folder_name} (Assigned Label: {label})")

            for img_name in os.listdir(folder_path):
                img_path = os.path.join(folder_path, img_name)

                if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    continue

                img = cv2.imread(img_path)
                if img is None:
                    continue

                feature = extract_features(img, part)

                if feature:
                    X.append(feature)
                    y.append(label)

    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.int32)

    if X.size == 0:
        print(f"❌ FATAL ERROR: Successfully loaded 0 samples from {path}. Check image files or Mediapipe detection.")
    else:
        # 💡 FIX: Check number of unique classes before returning
        unique_classes = np.unique(y)
        if len(unique_classes) < 2:
            print(f"❌ CRITICAL WARNING: Only {len(unique_classes)} class(es) loaded for {part} detection! Classes found: {unique_classes}")
            print("  -> This will cause the ValueError in SVC.fit(). Check your dataset folder names ('No_Yawn', 'Yawn') or structure.")

    print(f"✅ Successfully loaded {X.shape[0]} samples for {part} detection.")
    return X, y

# ----------------------------------------------------------------------
# ----- Step 3: Train Eye Blink Model (Fixed & Improved) -----
print("\n" + "="*50)
print("🔹 Training Eye Blink Model...")
X_eye, y_eye = load_dataset(extract_eye, part="eye")

if X_eye.shape[0] > 0 and len(np.unique(y_eye)) > 1:
    # Use random_state for reproducible split
    X_train, X_test, y_train, y_test = train_test_split(X_eye, y_eye, test_size=0.2, random_state=42)

    # IMPROVEMENT: Increase C value
    eye_model = SVC(kernel='linear', C=10.0, random_state=42)
    eye_model.fit(X_train, y_train)

    y_pred = eye_model.predict(X_test)
    print("\nEye Blink Accuracy:", accuracy_score(y_test, y_pred))
    joblib.dump(eye_model, "eye_blink_model.pkl")
    print("✅ Saved: eye_blink_model.pkl")
else:
    print("🛑 Skipping Eye Blink Model training due to insufficient data or classes.")

# ----------------------------------------------------------------------
# ----- Step 4: Train Yawn Model (Improved and Protected against ValueError) -----
print("\n" + "="*50)
print("🔹 Training Yawn Model...")
X_yawn, y_yawn = load_dataset(extract_yawn, part="mouth")

# 💡 FIX: Only attempt training if both data and multiple classes are present
if X_yawn.shape[0] > 0 and len(np.unique(y_yawn)) > 1:
    X_train, X_test, y_train, y_test = train_test_split(X_yawn, y_yawn, test_size=0.2, random_state=42)

    # Apply C=10.0 here as well for potential accuracy boost
    yawn_model = SVC(kernel='linear', C=10.0, random_state=42)
    yawn_model.fit(X_train, y_train)

    y_pred = yawn_model.predict(X_test)
    print("\nYawn Detection Accuracy:", accuracy_score(y_test, y_pred))
    joblib.dump(yawn_model, "yawn_model.pkl")
    print("✅ Saved: yawn_model.pkl")
else:
    print("🛑 Skipping Yawn Model training due to insufficient data or classes (Need at least 2 classes).")
print("="*50)

Directory already exists: /content/yawn_dataset
Directory already exists: /content/eye_blink_dataset
✅ Datasets extraction process complete.


🔹 Training Eye Blink Model...
  -> Eye labels: Closed=0, Open=1
❌ FATAL ERROR: Successfully loaded 0 samples from /content/eye_blink_dataset. Check image files or Mediapipe detection.
✅ Successfully loaded 0 samples for eye detection.
🛑 Skipping Eye Blink Model training due to insufficient data or classes.

🔹 Training Yawn Model...
  -> Yawn labels: No Yawn=0, Yawn=1
  -> Processing folder: yawn (Assigned Label: 1)
❌ CRITICAL WARNING: Only 1 class(es) loaded for mouth detection! Classes found: [1]
  -> This will cause the ValueError in SVC.fit(). Check your dataset folder names ('No_Yawn', 'Yawn') or structure.
✅ Successfully loaded 185 samples for mouth detection.
🛑 Skipping Yawn Model training due to insufficient data or classes (Need at least 2 classes).


In [ ]:
#!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 959.3 kB/s eta 0:00:00
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled p

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import os
import sys

# --- Configuration ---
DATA_DIR = "cnn_dataset"  # Directory created by the video generator (contains 0/ and 1/)
IMAGE_SIZE = (48, 48)     # Must match the size used in the generator script
BATCH_SIZE = 32
EPOCHS = 20               # Start with 20 epochs, you may need more
# Renamed and updated save path to reflect the "LFB model" theme requested by the user
LFB_MODEL_SAVE_PATH = "lfb_fatigue_detector_model.keras"

def load_data(data_directory, img_size, batch_size):
    """Loads and prepares the image dataset from the directory structure."""
    if not os.path.exists(data_directory):
        print(f"Error: Dataset directory '{data_directory}' not found.")
        print("Please run the 'video_to_cnn_dataset_generator.py' script first to generate the image data.")
        sys.exit(1)

    print("Loading data from disk...")

    # Using image_dataset_from_directory for easy loading of labeled data
    dataset = tf.keras.utils.image_dataset_from_directory(
        data_directory,
        labels='inferred',
        label_mode='binary',
        image_size=img_size,
        interpolation='bilinear',
        batch_size=batch_size,
        shuffle=True,
        seed=42  # for reproducible shuffling
    )

    # Normalize pixel values from [0, 255] to [0, 1]
    normalization_layer = tf.keras.layers.Rescaling(1./255)
    dataset = dataset.map(lambda x, y: (normalization_layer(x), y))

    # Determine dataset size for splitting (80% train, 20% validation)
    dataset_size = tf.data.experimental.cardinality(dataset).numpy()
    if dataset_size == 0:
        print(f"Error: No images found in '{data_directory}'. Check the folder structure.")
        sys.exit(1)

    train_size = int(0.8 * dataset_size)

    train_ds = dataset.take(train_size)
    val_ds = dataset.skip(train_size)

    print(f"Total batches found: {dataset_size}")
    print(f"Training batches: {tf.data.experimental.cardinality(train_ds).numpy()}")
    print(f"Validation batches: {tf.data.experimental.cardinality(val_ds).numpy()}")

    # Prefetch data for performance
    train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
    val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

    return train_ds, val_ds

def create_cnn_model(input_shape):
    """Defines a simple CNN architecture suitable for small eye images."""
    print("Defining LFB CNN model architecture...")
    model = Sequential([
        # First Conv/Pool block
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        # Second Conv/Pool block
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        # Third Conv/Pool block
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        # Classification layers
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # Binary classification (Alert or Fatigue)
    ])
    return model

def train_model(train_ds, val_ds):
    """Compiles and trains the defined CNN model."""

    # The input shape is (Height, Width, Color_Channels)
    input_shape = IMAGE_SIZE + (3,)
    model = create_cnn_model(input_shape)

    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    model.summary()
    print(f"Starting LFB model training for {EPOCHS} epochs...")

    # Train the model
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS
    )

    return model, history

if __name__ == "__main__":
    print(f"TensorFlow Version: {tf.__version__}")

    # Load and prepare data
    train_data, val_data = load_data(DATA_DIR, IMAGE_SIZE, BATCH_SIZE)

    # Train the model
    trained_model, training_history = train_model(train_data, val_data)

    # Save the model in the Keras format
    trained_model.save(LFB_MODEL_SAVE_PATH) # <-- Using the new save path
    print("\n---------------------------------------------------------")
    print(f"✅ LFB Model training completed. Model saved to: {os.path.abspath(LFB_MODEL_SAVE_PATH)}")
    print("---------------------------------------------------------")


In [ ]:
import cv2
import dlib
import numpy as np
import argparse
import os
import sys
import joblib
from scipy.spatial import distance as dist

# Attempt to import TensorFlow/Keras
try:
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Rescaling
    from tensorflow.keras.optimizers import Adam
    TENSORFLOW_AVAILABLE = True
except ImportError:
    print("Error: TensorFlow/Keras not found. Please run '!pip install tensorflow' first.")
    TENSORFLOW_AVAILABLE = False
    sys.exit(1)


# --- Configuration (Set Your Paths Here) ---
# NOTE: ⚠️ Change this path to your actual video file location
DEFAULT_VIDEO_PATH = "/content/drive/MyDrive/Video/9709794-uhd_3840_2160_25fps (1).mp4"
DLIB_PREDICTOR_PATH = "/content/drive/MyDrive/data_set/shape_predictor_68_face_landmarks.dat"

# Training Output Configuration
DATA_DIR = "cnn_dataset"  # Output folder for generated images (0/ and 1/)
LFB_MODEL_SAVE_PATH = "lfb_fatigue_detector_model.keras"
IMAGE_SIZE = (48, 48)     # Target size for CNN input
BATCH_SIZE = 32
EPOCHS = 20

# Feature Calculation Constants
EAR_THRESHOLD = 0.23  # More sensitive threshold for fatigue (0.25 is too high for some videos)
ALERT_LABEL = 0
FATIGUE_LABEL = 1
FACE_LANDMARKS = list(range(68))
EYE_CROP_INDICES = list(range(36, 48))
FRAME_SKIP_RATE = 5   # Process only 1 in 5 frames


# ----------------------------------------------
# --- 1. DATA GENERATION FUNCTIONS (Video -> Images) ---
# ----------------------------------------------

def eye_aspect_ratio(eye):
    """Calculates the Eye Aspect Ratio (EAR)."""
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

def get_ear_value(shape):
    """Calculates the average EAR from dlib shape object."""
    landmarks = np.array([(shape.part(i).x, shape.part(i).y) for i in FACE_LANDMARKS])
    left_eye = landmarks[list(range(42, 48))]
    right_eye = landmarks[list(range(36, 42))]
    ear_left = eye_aspect_ratio(left_eye)
    ear_right = eye_aspect_ratio(right_eye)
    return (ear_left + ear_right) / 2.0, landmarks

def generate_cnn_dataset(video_path, predictor_path, output_dir):
    """Processes video frames to create a labeled image dataset (0/ and 1/)."""
    print(f"\n--- 1.1. Starting Image Preparation from: {video_path} ---")

    if not os.path.exists(predictor_path):
        print(f"Error: Dlib predictor not found at {predictor_path}. Cannot generate dataset.")
        return False
    if not os.path.exists(video_path):
        print(f"Error: Video file not found at {video_path}. Cannot generate dataset.")
        return False

    os.makedirs(os.path.join(output_dir, str(ALERT_LABEL)), exist_ok=True)
    os.makedirs(os.path.join(output_dir, str(FATIGUE_LABEL)), exist_ok=True)

    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(predictor_path)
    cap = cv2.VideoCapture(video_path)

    frame_count, alert_count, fatigue_count = 0, 0, 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # Only process a fraction of frames to speed up training and reduce correlation
        if frame_count % FRAME_SKIP_RATE != 0:
            continue

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray, 0)

        if len(faces) > 0:
            rect = faces[0]
            shape = predictor(gray, rect)
            ear_avg, landmarks = get_ear_value(shape)

            # Determine label: EAR below threshold is FATIGUE (1)
            current_label = FATIGUE_LABEL if ear_avg < EAR_THRESHOLD else ALERT_LABEL

            # --- Cropping Logic around the eyes ---
            points_to_crop = landmarks[EYE_CROP_INDICES]
            x_min = np.min(points_to_crop[:, 0]) - 15
            x_max = np.max(points_to_crop[:, 0]) + 15
            y_min = np.min(points_to_crop[:, 1]) - 15
            y_max = np.max(points_to_crop[:, 1]) + 15

            # Clamp coordinates to frame bounds
            x_min = max(0, x_min); y_min = max(0, y_min)
            x_max = min(frame.shape[1], x_max); y_max = min(frame.shape[0], y_max)

            cropped_image = frame[y_min:y_max, x_min:x_max]

            if cropped_image.size != 0:
                resized_image = cv2.resize(cropped_image, IMAGE_SIZE)
                final_image = resized_image # Use BGR image for CNN input (3 channels)

                # Save the image
                label_dir = os.path.join(output_dir, str(current_label))
                image_filename = f"frame_{frame_count:06d}_{ear_avg:.3f}.jpg"
                save_path = os.path.join(label_dir, image_filename)
                cv2.imwrite(save_path, final_image)

                if current_label == ALERT_LABEL:
                    alert_count += 1
                else:
                    fatigue_count += 1

        if frame_count % 500 == 0:
            print(f"Processed {frame_count} frames. Alert saved: {alert_count}, Fatigue saved: {fatigue_count}")

    cap.release()
    print("\n---------------------------------------------------------")
    print(f"✅ Image dataset preparation finished.")
    print(f"Total images saved: Alert={alert_count}, Fatigue={fatigue_count}")
    print("---------------------------------------------------------")

    # Check if we actually generated any fatigue images
    if fatigue_count == 0 and alert_count == 0:
        print("Error: No images were generated. Check video path and dlib predictor.")
        return False
    elif fatigue_count == 0:
        print("Warning: Only ALERT (0) images were generated. CNN training results will be poor.")
        print("Try using a video with more eye closure/blinks, or lower the EAR_THRESHOLD.")

    return True


# ----------------------------------------------
# --- 2. MODEL TRAINING FUNCTIONS (Images -> Model) ---
# ----------------------------------------------

def load_data(data_directory, img_size, batch_size):
    """Loads and prepares the image dataset from the directory structure."""
    print("\n--- 2.1. Loading Data from Disk ---")

    dataset = tf.keras.utils.image_dataset_from_directory(
        data_directory,
        labels='inferred',
        label_mode='binary', # Output is 0 or 1
        image_size=img_size,
        interpolation='bilinear',
        batch_size=batch_size,
        shuffle=True,
        seed=42
    )

    # Normalize pixel values from [0, 255] to [0, 1]
    normalization_layer = Rescaling(1./255)
    dataset = dataset.map(lambda x, y: (normalization_layer(x), y))

    dataset_size = tf.data.experimental.cardinality(dataset).numpy()
    if dataset_size == 0:
        print(f"Error: No batches found in '{data_directory}'. Cannot train LFB model.")
        sys.exit(1)

    # Split dataset: 80% train, 20% validation
    train_size = int(0.8 * dataset_size)
    train_ds = dataset.take(train_size)
    val_ds = dataset.skip(train_size)

    print(f"Total batches found: {dataset_size}. Split: Train={train_size}, Validation={dataset_size - train_size}")

    train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
    val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

    return train_ds, val_ds

def create_cnn_model(input_shape):
    """Defines the LFB CNN model architecture."""
    print("--- 2.2. Defining LFB CNN Model Architecture ---")
    model = Sequential([
        # First Block
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        # Second Block
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        # Third Block
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        # Classification layers
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # Binary output
    ])
    return model

def train_lfb_model(train_ds, val_ds):
    """Compiles and trains the defined LFB CNN model."""

    input_shape = IMAGE_SIZE + (3,)
    model = create_cnn_model(input_shape)

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    model.summary()
    print(f"--- 2.3. Starting LFB Model Training for {EPOCHS} epochs ---")

    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS
    )

    return model

# ----------------------------------------------
# --- 3. MAIN EXECUTION ---
# ----------------------------------------------

if __name__ == "__main__":

    # ⚠️ Set up the arguments and path. This allows running it from command line,
    # but defaults to the hardcoded path above.
    parser = argparse.ArgumentParser(description="Full LFB Drowsiness Detection Training Pipeline.")
    parser.add_argument("--video", type=str, default=DEFAULT_VIDEO_PATH,
                        help="Path to the input video file.")
    parser.add_argument("--predictor", type=str, default=DLIB_PREDICTOR_PATH,
                        help="Path to the dlib shape predictor file.")

    args = parser.parse_args()

    # --- Step 1: Data Generation ---
    success = generate_cnn_dataset(args.video, args.predictor, DATA_DIR)

    if not success:
        print("\nPipeline failed at the data generation step. Please check the video and predictor paths.")
        sys.exit(1)

    # --- Step 2: Model Training ---
    try:
        # Load and prepare data
        train_data, val_data = load_data(DATA_DIR, IMAGE_SIZE, BATCH_SIZE)

        # Train the model
        trained_model = train_lfb_model(train_data, val_data)

        # Save the model
        trained_model.save(LFB_MODEL_SAVE_PATH)

        print("\n---------------------------------------------------------")
        print(f"✅ FINAL SUCCESS: LFB Model saved to: {os.path.abspath(LFB_MODEL_SAVE_PATH)}")
        print("---------------------------------------------------------")

    except Exception as e:
        print(f"\nError during model training (Step 2): {e}")
        print("Please verify the 'cnn_dataset' folder structure.")
        sys.exit(1)


In [ ]:
import cv2
import dlib
import numpy as np
import argparse
import os
import sys
import joblib
from scipy.spatial import distance as dist

# Attempt to import TensorFlow/Keras
try:
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Rescaling
    from tensorflow.keras.optimizers import Adam
    TENSORFLOW_AVAILABLE = True
except ImportError:
    print("Error: TensorFlow/Keras not found. Please run '!pip install tensorflow' first.")
    TENSORFLOW_AVAILABLE = False
    sys.exit(1)


# --- Configuration (Set Your Paths Here) ---
# NOTE: ⚠️ Change this path to your actual video file location
DEFAULT_VIDEO_PATH = "/content/drive/MyDrive/Video/Screen-Recording (5).mp4"
DLIB_PREDICTOR_PATH = "/content/drive/MyDrive/data_set/shape_predictor_68_face_landmarks.dat"

# Training Output Configuration
DATA_DIR = "cnn_dataset"  # Output folder for generated images (0/ and 1/)
LFB_MODEL_SAVE_PATH = "lfb_fatigue_detector_model.keras"
IMAGE_SIZE = (48, 48)     # Target size for CNN input
BATCH_SIZE = 32
EPOCHS = 20

# Feature Calculation Constants
EAR_THRESHOLD = 0.23  # More sensitive threshold for fatigue (0.25 is too high for some videos)
ALERT_LABEL = 0
FATIGUE_LABEL = 1
FACE_LANDMARKS = list(range(68))
EYE_CROP_INDICES = list(range(36, 48))
FRAME_SKIP_RATE = 5   # Process only 1 in 5 frames


# ----------------------------------------------
# --- 1. DATA GENERATION FUNCTIONS (Video -> Images) ---
# ----------------------------------------------

def eye_aspect_ratio(eye):
    """Calculates the Eye Aspect Ratio (EAR)."""
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

def get_ear_value(shape):
    """Calculates the average EAR from dlib shape object."""
    landmarks = np.array([(shape.part(i).x, shape.part(i).y) for i in FACE_LANDMARKS])
    left_eye = landmarks[list(range(42, 48))]
    right_eye = landmarks[list(range(36, 42))]
    ear_left = eye_aspect_ratio(left_eye)
    ear_right = eye_aspect_ratio(right_eye)
    return (ear_left + ear_right) / 2.0, landmarks

def generate_cnn_dataset(video_path, predictor_path, output_dir):
    """Processes video frames to create a labeled image dataset (0/ and 1/)."""
    print(f"\n--- 1.1. Starting Image Preparation from: {video_path} ---")

    if not os.path.exists(predictor_path):
        print(f"Error: Dlib predictor not found at {predictor_path}. Cannot generate dataset.")
        return False
    if not os.path.exists(video_path):
        print(f"Error: Video file not found at {video_path}. Cannot generate dataset.")
        return False

    os.makedirs(os.path.join(output_dir, str(ALERT_LABEL)), exist_ok=True)
    os.makedirs(os.path.join(output_dir, str(FATIGUE_LABEL)), exist_ok=True)

    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(predictor_path)
    cap = cv2.VideoCapture(video_path)

    frame_count, alert_count, fatigue_count = 0, 0, 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # Only process a fraction of frames to speed up training and reduce correlation
        if frame_count % FRAME_SKIP_RATE != 0:
            continue

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray, 0)

        if len(faces) > 0:
            rect = faces[0]
            shape = predictor(gray, rect)
            ear_avg, landmarks = get_ear_value(shape)

            # Determine label: EAR below threshold is FATIGUE (1)
            current_label = FATIGUE_LABEL if ear_avg < EAR_THRESHOLD else ALERT_LABEL

            # --- Cropping Logic around the eyes ---
            points_to_crop = landmarks[EYE_CROP_INDICES]
            x_min = np.min(points_to_crop[:, 0]) - 15
            x_max = np.max(points_to_crop[:, 0]) + 15
            y_min = np.min(points_to_crop[:, 1]) - 15
            y_max = np.max(points_to_crop[:, 1]) + 15

            # Clamp coordinates to frame bounds
            x_min = max(0, x_min); y_min = max(0, y_min)
            x_max = min(frame.shape[1], x_max); y_max = min(frame.shape[0], y_max)

            cropped_image = frame[y_min:y_max, x_min:x_max]

            if cropped_image.size != 0:
                resized_image = cv2.resize(cropped_image, IMAGE_SIZE)
                final_image = resized_image # Use BGR image for CNN input (3 channels)

                # Save the image
                label_dir = os.path.join(output_dir, str(current_label))
                image_filename = f"frame_{frame_count:06d}_{ear_avg:.3f}.jpg"
                save_path = os.path.join(label_dir, image_filename)
                cv2.imwrite(save_path, final_image)

                if current_label == ALERT_LABEL:
                    alert_count += 1
                else:
                    fatigue_count += 1

        if frame_count % 500 == 0:
            print(f"Processed {frame_count} frames. Alert saved: {alert_count}, Fatigue saved: {fatigue_count}")

    cap.release()
    print("\n---------------------------------------------------------")
    print(f"✅ Image dataset preparation finished.")
    print(f"Total images saved: Alert={alert_count}, Fatigue={fatigue_count}")
    print("---------------------------------------------------------")

    # Check if we actually generated any fatigue images
    if fatigue_count == 0 and alert_count == 0:
        print("Error: No images were generated. Check video path and dlib predictor.")
        return False
    elif fatigue_count == 0:
        print("Warning: Only ALERT (0) images were generated. CNN training results will be poor.")
        print("Try using a video with more eye closure/blinks, or lower the EAR_THRESHOLD.")

    return True


# ----------------------------------------------
# --- 2. MODEL TRAINING FUNCTIONS (Images -> Model) ---
# ----------------------------------------------

def load_data(data_directory, img_size, batch_size):
    """Loads and prepares the image dataset from the directory structure."""
    print("\n--- 2.1. Loading Data from Disk ---")

    dataset = tf.keras.utils.image_dataset_from_directory(
        data_directory,
        labels='inferred',
        label_mode='binary', # Output is 0 or 1
        image_size=img_size,
        interpolation='bilinear',
        batch_size=batch_size,
        shuffle=True,
        seed=42
    )

    # Normalize pixel values from [0, 255] to [0, 1]
    normalization_layer = Rescaling(1./255)
    dataset = dataset.map(lambda x, y: (normalization_layer(x), y))

    dataset_size = tf.data.experimental.cardinality(dataset).numpy()
    if dataset_size == 0:
        print(f"Error: No batches found in '{data_directory}'. Cannot train LFB model.")
        sys.exit(1)

    # Split dataset: 80% train, 20% validation
    train_size = int(0.8 * dataset_size)
    train_ds = dataset.take(train_size)
    val_ds = dataset.skip(train_size)

    print(f"Total batches found: {dataset_size}. Split: Train={train_size}, Validation={dataset_size - train_size}")

    train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
    val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

    return train_ds, val_ds

def create_cnn_model(input_shape):
    """Defines the LFB CNN model architecture."""
    print("--- 2.2. Defining LFB CNN Model Architecture ---")
    model = Sequential([
        # First Block
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        # Second Block
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        # Third Block
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        # Classification layers
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # Binary output
    ])
    return model

def train_lfb_model(train_ds, val_ds):
    """Compiles and trains the defined LFB CNN model."""

    input_shape = IMAGE_SIZE + (3,)
    model = create_cnn_model(input_shape)

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    model.summary()
    print(f"--- 2.3. Starting LFB Model Training for {EPOCHS} epochs ---")

    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS
    )

    return model

# ----------------------------------------------
# --- 3. MAIN EXECUTION ---
# ----------------------------------------------

if __name__ == "__main__":

    # FIX: Remove the Jupyter/Colab kernel arguments (-f <filename>)
    # that cause the 'unrecognized arguments' error in argparse.
    if '-f' in sys.argv:
        f_index = sys.argv.index('-f')
        # Remove the '-f' flag and the following argument (the json file path)
        sys.argv = sys.argv[:f_index] + sys.argv[f_index+2:]

    # ⚠️ Set up the arguments and path. This allows running it from command line,
    # but defaults to the hardcoded path above.
    parser = argparse.ArgumentParser(description="Full LFB Drowsiness Detection Training Pipeline.")
    parser.add_argument("--video", type=str, default=DEFAULT_VIDEO_PATH,
                        help="Path to the input video file.")
    parser.add_argument("--predictor", type=str, default=DLIB_PREDICTOR_PATH,
                        help="Path to the dlib shape predictor file.")

    args = parser.parse_args()

    # --- Step 1: Data Generation ---
    success = generate_cnn_dataset(args.video, args.predictor, DATA_DIR)

    if not success:
        print("\nPipeline failed at the data generation step. Please check the video and predictor paths.")
        sys.exit(1)

    # --- Step 2: Model Training ---
    try:
        # Load and prepare data
        train_data, val_data = load_data(DATA_DIR, IMAGE_SIZE, BATCH_SIZE)

        # Train the model
        trained_model = train_lfb_model(train_data, val_data)

        # Save the model
        trained_model.save(LFB_MODEL_SAVE_PATH)

        print("\n---------------------------------------------------------")
        print(f"✅ FINAL SUCCESS: LFB Model saved to: {os.path.abspath(LFB_MODEL_SAVE_PATH)}")
        print("---------------------------------------------------------")

    except Exception as e:
        print(f"\nError during model training (Step 2): {e}")
        print("Please verify the 'cnn_dataset' folder structure.")
        sys.exit(1)


In [ ]:
import cv2
import dlib
import numpy as np
import argparse
import os
import sys
import joblib
from scipy.spatial import distance as dist

# Attempt to import TensorFlow/Keras
try:
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Rescaling
    from tensorflow.keras.optimizers import Adam
    TENSORFLOW_AVAILABLE = True
except ImportError:
    print("Error: TensorFlow/Keras not found. Please run '!pip install tensorflow' first.")
    TENSORFLOW_AVAILABLE = False
    sys.exit(1)


# --- Configuration (Set Your Paths Here) ---
# NOTE: ⚠️ Change this path to your actual video file location
DEFAULT_VIDEO_PATH = "/content/drive/MyDrive/Video/Screen-Recording (5).mp4"
DLIB_PREDICTOR_PATH = "/content/drive/MyDrive/data_set/shape_predictor_68_face_landmarks.dat"

# Training Output Configuration
DATA_DIR = "cnn_dataset"  # Output folder for generated images (0/ and 1/)
LFB_MODEL_SAVE_PATH = "lfb_fatigue_detector_model.keras"
LFB_MODEL_YAML_PATH = "lfb_model_architecture.yaml" # New YAML save path
IMAGE_SIZE = (48, 48)     # Target size for CNN input
BATCH_SIZE = 32
EPOCHS = 20

# Feature Calculation Constants
EAR_THRESHOLD = 0.23  # More sensitive threshold for fatigue (0.25 is too high for some videos)
ALERT_LABEL = 0
FATIGUE_LABEL = 1
FACE_LANDMARKS = list(range(68))
EYE_CROP_INDICES = list(range(36, 48))
FRAME_SKIP_RATE = 5   # Process only 1 in 5 frames


# ----------------------------------------------
# --- 1. DATA GENERATION FUNCTIONS (Video -> Images) ---
# ----------------------------------------------

def eye_aspect_ratio(eye):
    """Calculates the Eye Aspect Ratio (EAR)."""
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

def get_ear_value(shape):
    """Calculates the average EAR from dlib shape object."""
    landmarks = np.array([(shape.part(i).x, shape.part(i).y) for i in FACE_LANDMARKS])
    left_eye = landmarks[list(range(42, 48))]
    right_eye = landmarks[list(range(36, 42))]
    ear_left = eye_aspect_ratio(left_eye)
    ear_right = eye_aspect_ratio(right_eye)
    return (ear_left + ear_right) / 2.0, landmarks

def generate_cnn_dataset(video_path, predictor_path, output_dir):
    """Processes video frames to create a labeled image dataset (0/ and 1/)."""
    print(f"\n--- 1.1. Starting Image Preparation from: {video_path} ---")

    if not os.path.exists(predictor_path):
        print(f"Error: Dlib predictor not found at {predictor_path}. Cannot generate dataset.")
        return False
    if not os.path.exists(video_path):
        print(f"Error: Video file not found at {video_path}. Cannot generate dataset.")
        return False

    os.makedirs(os.path.join(output_dir, str(ALERT_LABEL)), exist_ok=True)
    os.makedirs(os.path.join(output_dir, str(FATIGUE_LABEL)), exist_ok=True)

    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(predictor_path)
    cap = cv2.VideoCapture(video_path)

    frame_count, alert_count, fatigue_count = 0, 0, 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # Only process a fraction of frames to speed up training and reduce correlation
        if frame_count % FRAME_SKIP_RATE != 0:
            continue

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray, 0)

        if len(faces) > 0:
            rect = faces[0]
            shape = predictor(gray, rect)
            ear_avg, landmarks = get_ear_value(shape)

            # Determine label: EAR below threshold is FATIGUE (1)
            current_label = FATIGUE_LABEL if ear_avg < EAR_THRESHOLD else ALERT_LABEL

            # --- Cropping Logic around the eyes ---
            points_to_crop = landmarks[EYE_CROP_INDICES]
            x_min = np.min(points_to_crop[:, 0]) - 15
            x_max = np.max(points_to_crop[:, 0]) + 15
            y_min = np.min(points_to_crop[:, 1]) - 15
            y_max = np.max(points_to_crop[:, 1]) + 15

            # Clamp coordinates to frame bounds
            x_min = max(0, x_min); y_min = max(0, y_min)
            x_max = min(frame.shape[1], x_max); y_max = min(frame.shape[0], y_max)

            cropped_image = frame[y_min:y_max, x_min:x_max]

            if cropped_image.size != 0:
                resized_image = cv2.resize(cropped_image, IMAGE_SIZE)
                final_image = resized_image # Use BGR image for CNN input (3 channels)

                # Save the image
                label_dir = os.path.join(output_dir, str(current_label))
                image_filename = f"frame_{frame_count:06d}_{ear_avg:.3f}.jpg"
                save_path = os.path.join(label_dir, image_filename)
                cv2.imwrite(save_path, final_image)

                if current_label == ALERT_LABEL:
                    alert_count += 1
                else:
                    fatigue_count += 1

        if frame_count % 500 == 0:
            print(f"Processed {frame_count} frames. Alert saved: {alert_count}, Fatigue saved: {fatigue_count}")

    cap.release()
    print("\n---------------------------------------------------------")
    print(f"✅ Image dataset preparation finished.")
    print(f"Total images saved: Alert={alert_count}, Fatigue={fatigue_count}")
    print("---------------------------------------------------------")

    # Check if we actually generated any fatigue images
    if fatigue_count == 0 and alert_count == 0:
        print("Error: No images were generated. Check video path and dlib predictor.")
        return False
    elif fatigue_count == 0:
        print("Warning: Only ALERT (0) images were generated. CNN training results will be poor.")
        print("Try using a video with more eye closure/blinks, or lower the EAR_THRESHOLD.")

    return True


# ----------------------------------------------
# --- 2. MODEL TRAINING FUNCTIONS (Images -> Model) ---
# ----------------------------------------------

def load_data(data_directory, img_size, batch_size):
    """Loads and prepares the image dataset from the directory structure."""
    print("\n--- 2.1. Loading Data from Disk ---")

    dataset = tf.keras.utils.image_dataset_from_directory(
        data_directory,
        labels='inferred',
        label_mode='binary', # Output is 0 or 1
        image_size=img_size,
        interpolation='bilinear',
        batch_size=batch_size,
        shuffle=True,
        seed=42
    )

    # Normalize pixel values from [0, 255] to [0, 1]
    normalization_layer = Rescaling(1./255)
    dataset = dataset.map(lambda x, y: (normalization_layer(x), y))

    dataset_size = tf.data.experimental.cardinality(dataset).numpy()
    if dataset_size == 0:
        print(f"Error: No batches found in '{data_directory}'. Cannot train LFB model.")
        sys.exit(1)

    # Split dataset: 80% train, 20% validation
    train_size = int(0.8 * dataset_size)
    train_ds = dataset.take(train_size)
    val_ds = dataset.skip(train_size)

    print(f"Total batches found: {dataset_size}. Split: Train={train_size}, Validation={dataset_size - train_size}")

    train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
    val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

    return train_ds, val_ds

def create_cnn_model(input_shape):
    """Defines the LFB CNN model architecture."""
    print("--- 2.2. Defining LFB CNN Model Architecture ---")
    model = Sequential([
        # First Block
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        # Second Block
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        # Third Block
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        # Classification layers
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # Binary output
    ])
    return model

def train_lfb_model(train_ds, val_ds):
    """Compiles and trains the defined LFB CNN model."""

    input_shape = IMAGE_SIZE + (3,)
    model = create_cnn_model(input_shape)

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    model.summary()
    print(f"--- 2.3. Starting LFB Model Training for {EPOCHS} epochs ---")

    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS
    )

    return model

# ----------------------------------------------
# --- 3. MAIN EXECUTION ---
# ----------------------------------------------

if __name__ == "__main__":

    # FIX: Remove the Jupyter/Colab kernel arguments (-f <filename>)
    # that cause the 'unrecognized arguments' error in argparse.
    if '-f' in sys.argv:
        f_index = sys.argv.index('-f')
        # Remove the '-f' flag and the following argument (the json file path)
        sys.argv = sys.argv[:f_index] + sys.argv[f_index+2:]

    # ⚠️ Set up the arguments and path. This allows running it from command line,
    # but defaults to the hardcoded path above.
    parser = argparse.ArgumentParser(description="Full LFB Drowsiness Detection Training Pipeline.")
    parser.add_argument("--video", type=str, default=DEFAULT_VIDEO_PATH,
                        help="Path to the input video file.")
    parser.add_argument("--predictor", type=str, default=DLIB_PREDICTOR_PATH,
                        help="Path to the dlib shape predictor file.")

    args = parser.parse_args()

    # --- Step 1: Data Generation ---
    success = generate_cnn_dataset(args.video, args.predictor, DATA_DIR)

    if not success:
        print("\nPipeline failed at the data generation step. Please check the video and predictor paths.")
        sys.exit(1)

    # --- Step 2: Model Training ---
    try:
        # Load and prepare data
        train_data, val_data = load_data(DATA_DIR, IMAGE_SIZE, BATCH_SIZE)

        # Train the model
        trained_model = train_lfb_model(train_data, val_data)

        # --- Save the model (FULL MODEL) ---
        trained_model.save(LFB_MODEL_SAVE_PATH)

        # --- Save the architecture as YAML (NEW) ---
        # NOTE: This requires pyyaml to be installed, which is usually included in Colab/Jupyter.
        # If running locally, you might need to install it: `pip install pyyaml`
        yaml_model = trained_model.to_yaml()
        with open(LFB_MODEL_YAML_PATH, "w") as yaml_file:
            yaml_file.write(yaml_model)

        print("\n---------------------------------------------------------")
        print(f"✅ FINAL SUCCESS: LFB Model (Full) saved to: {os.path.abspath(LFB_MODEL_SAVE_PATH)}")
        print(f"✅ LFB Model (YAML Architecture) saved to: {os.path.abspath(LFB_MODEL_YAML_PATH)}")
        print("---------------------------------------------------------")

    except Exception as e:
        print(f"\nError during model training (Step 2): {e}")
        print("Please verify the 'cnn_dataset' folder structure.")
        sys.exit(1)


ModuleNotFoundError: No module named 'numpy.strings'

In [ ]:
import cv2
import dlib
import numpy as np
import argparse
import os
import sys
import joblib
from scipy.spatial import distance as dist

# Attempt to import TensorFlow/Keras
try:
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    # REMOVED: model_to_yaml causes import errors in some Keras versions.
    # We will use the more stable built-in 'trained_model.to_json()' method instead.
    from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Rescaling
    from tensorflow.keras.optimizers import Adam
    TENSORFLOW_AVAILABLE = True
except ImportError:
    print("Error: TensorFlow/Keras not found. Please run '!pip install tensorflow' first.")
    TENSORFLOW_AVAILABLE = False
    sys.exit(1)


# --- Configuration (Set Your Paths Here) ---
# NOTE: ⚠️ Path updated based on your input
DEFAULT_VIDEO_PATH = "/content/drive/MyDrive/Video/Screen-Recording (5).mp4"
DLIB_PREDICTOR_PATH = "/content/drive/MyDrive/data_set/shape_predictor_68_face_landmarks.dat"

# Training Output Configuration
DATA_DIR = "cnn_dataset"  # Output folder for generated images (0/ and 1/)
LFB_MODEL_SAVE_PATH = "lfb_fatigue_detector_model.keras"
# CHANGED: Switched from YAML to JSON for robust architecture serialization
LFB_MODEL_JSON_PATH = "lfb_model_architecture.json"
IMAGE_SIZE = (48, 48)     # Target size for CNN input
BATCH_SIZE = 32
EPOCHS = 20

# Feature Calculation Constants
EAR_THRESHOLD = 0.23  # More sensitive threshold for fatigue (0.25 is too high for some videos)
ALERT_LABEL = 0
FATIGUE_LABEL = 1
FACE_LANDMARKS = list(range(68))
EYE_CROP_INDICES = list(range(36, 48))
FRAME_SKIP_RATE = 5   # Process only 1 in 5 frames


# ----------------------------------------------
# --- 1. DATA GENERATION FUNCTIONS (Video -> Images) ---
# ----------------------------------------------

def eye_aspect_ratio(eye):
    """Calculates the Eye Aspect Ratio (EAR)."""
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

def get_ear_value(shape):
    """Calculates the average EAR from dlib shape object."""
    landmarks = np.array([(shape.part(i).x, shape.part(i).y) for i in FACE_LANDMARKS])
    left_eye = landmarks[list(range(42, 48))]
    right_eye = landmarks[list(range(36, 42))]
    ear_left = eye_aspect_ratio(left_eye)
    ear_right = eye_aspect_ratio(right_eye)
    return (ear_left + ear_right) / 2.0, landmarks

def generate_cnn_dataset(video_path, predictor_path, output_dir):
    """Processes video frames to create a labeled image dataset (0/ and 1/)."""
    print(f"\n--- 1.1. Starting Image Preparation from: {video_path} ---")

    if not os.path.exists(predictor_path):
        print(f"Error: Dlib predictor not found at {predictor_path}. Cannot generate dataset.")
        return False
    if not os.path.exists(video_path):
        print(f"Error: Video file not found at {video_path}. Cannot generate dataset.")
        return False

    os.makedirs(os.path.join(output_dir, str(ALERT_LABEL)), exist_ok=True)
    os.makedirs(os.path.join(output_dir, str(FATIGUE_LABEL)), exist_ok=True)

    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(predictor_path)
    cap = cv2.VideoCapture(video_path)

    frame_count, alert_count, fatigue_count = 0, 0, 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # Only process a fraction of frames to speed up training and reduce correlation
        if frame_count % FRAME_SKIP_RATE != 0:
            continue

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray, 0)

        if len(faces) > 0:
            rect = faces[0]
            shape = predictor(gray, rect)
            ear_avg, landmarks = get_ear_value(shape)

            # Determine label: EAR below threshold is FATIGUE (1)
            current_label = FATIGUE_LABEL if ear_avg < EAR_THRESHOLD else ALERT_LABEL

            # --- Cropping Logic around the eyes ---
            points_to_crop = landmarks[EYE_CROP_INDICES]
            x_min = np.min(points_to_crop[:, 0]) - 15
            x_max = np.max(points_to_crop[:, 0]) + 15
            y_min = np.min(points_to_crop[:, 1]) - 15
            y_max = np.max(points_to_crop[:, 1]) + 15

            # Clamp coordinates to frame bounds
            x_min = max(0, x_min); y_min = max(0, y_min)
            x_max = min(frame.shape[1], x_max); y_max = min(frame.shape[0], y_max)

            cropped_image = frame[y_min:y_max, x_min:x_max]

            if cropped_image.size != 0:
                resized_image = cv2.resize(cropped_image, IMAGE_SIZE)
                final_image = resized_image # Use BGR image for CNN input (3 channels)

                # Save the image
                label_dir = os.path.join(output_dir, str(current_label))
                image_filename = f"frame_{frame_count:06d}_{ear_avg:.3f}.jpg"
                save_path = os.path.join(label_dir, image_filename)
                cv2.imwrite(save_path, final_image)

                if current_label == ALERT_LABEL:
                    alert_count += 1
                else:
                    fatigue_count += 1

        if frame_count % 500 == 0:
            print(f"Processed {frame_count} frames. Alert saved: {alert_count}, Fatigue saved: {fatigue_count}")

    cap.release()
    print("\n---------------------------------------------------------")
    print(f"✅ Image dataset preparation finished.")
    print(f"Total images saved: Alert={alert_count}, Fatigue={fatigue_count}")
    print("---------------------------------------------------------")

    # Check if we actually generated any fatigue images
    if fatigue_count == 0 and alert_count == 0:
        print("Error: No images were generated. Check video path and dlib predictor.")
        return False
    elif fatigue_count == 0:
        print("Warning: Only ALERT (0) images were generated. CNN training results will be poor.")
        print("Try using a video with more eye closure/blinks, or lower the EAR_THRESHOLD.")

    return True


# ----------------------------------------------
# --- 2. MODEL TRAINING FUNCTIONS (Images -> Model) ---
# ----------------------------------------------

def load_data(data_directory, img_size, batch_size):
    """Loads and prepares the image dataset from the directory structure."""
    print("\n--- 2.1. Loading Data from Disk ---")

    dataset = tf.keras.utils.image_dataset_from_directory(
        data_directory,
        labels='inferred',
        label_mode='binary', # Output is 0 or 1
        image_size=img_size,
        interpolation='bilinear',
        batch_size=batch_size,
        shuffle=True,
        seed=42
    )

    # Normalize pixel values from [0, 255] to [0, 1]
    normalization_layer = Rescaling(1./255)
    dataset = dataset.map(lambda x, y: (normalization_layer(x), y))

    dataset_size = tf.data.experimental.cardinality(dataset).numpy()
    if dataset_size == 0:
        print(f"Error: No batches found in '{data_directory}'. Cannot train LFB model.")
        sys.exit(1)

    # Split dataset: 80% train, 20% validation
    train_size = int(0.8 * dataset_size)
    train_ds = dataset.take(train_size)
    val_ds = dataset.skip(train_size)

    print(f"Total batches found: {dataset_size}. Split: Train={train_size}, Validation={dataset_size - train_size}")

    train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
    val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

    return train_ds, val_ds

def create_cnn_model(input_shape):
    """Defines the LFB CNN model architecture."""
    print("--- 2.2. Defining LFB CNN Model Architecture ---")
    model = Sequential([
        # First Block
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        # Second Block
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        # Third Block
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        # Classification layers
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # Binary output
    ])
    return model

def train_lfb_model(train_ds, val_ds):
    """Compiles and trains the defined LFB CNN model."""

    input_shape = IMAGE_SIZE + (3,)
    model = create_cnn_model(input_shape)

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    model.summary()
    print(f"--- 2.3. Starting LFB Model Training for {EPOCHS} epochs ---")

    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS
    )

    return model

# ----------------------------------------------
# --- 3. MAIN EXECUTION ---
# ----------------------------------------------

if __name__ == "__main__":

    # FIX: Remove the Jupyter/Colab kernel arguments (-f <filename>)
    # that cause the 'unrecognized arguments' error in argparse.
    if '-f' in sys.argv:
        f_index = sys.argv.index('-f')
        # Remove the '-f' flag and the following argument (the json file path)
        sys.argv = sys.argv[:f_index] + sys.argv[f_index+2:]

    # ⚠️ Set up the arguments and path. This allows running it from command line,
    # but defaults to the hardcoded path above.
    parser = argparse.ArgumentParser(description="Full LFB Drowsiness Detection Training Pipeline.")
    parser.add_argument("--video", type=str, default=DEFAULT_VIDEO_PATH,
                        help="Path to the input video file.")
    parser.add_argument("--predictor", type=str, default=DLIB_PREDICTOR_PATH,
                        help="Path to the dlib shape predictor file.")

    args = parser.parse_args()

    # --- Step 1: Data Generation ---
    success = generate_cnn_dataset(args.video, args.predictor, DATA_DIR)

    if not success:
        print("\nPipeline failed at the data generation step. Please check the video and predictor paths.")
        sys.exit(1)

    # --- Step 2: Model Training ---
    try:
        # Load and prepare data
        train_data, val_data = load_data(DATA_DIR, IMAGE_SIZE, BATCH_SIZE)

        # Train the model
        trained_model = train_lfb_model(train_data, val_data)

        # --- Save the model (FULL MODEL) ---
        trained_model.save(LFB_MODEL_SAVE_PATH)

        # --- Save the architecture as JSON (FIX) ---
        # The .to_json() method is generally available on the Sequential model object.
        json_model = trained_model.to_json()
        with open(LFB_MODEL_JSON_PATH, "w") as json_file:
            json_file.write(json_model)

        print("\n---------------------------------------------------------")
        print(f"✅ FINAL SUCCESS: LFB Model (Full) saved to: {os.path.abspath(LFB_MODEL_SAVE_PATH)}")
        print(f"✅ LFB Model (JSON Architecture) saved to: {os.path.abspath(LFB_MODEL_JSON_PATH)}")
        print("---------------------------------------------------------")

    except Exception as e:
        print(f"\nError during model training (Step 2): {e}")
        print("Please verify the 'cnn_dataset' folder structure.")
        sys.exit(1)


ModuleNotFoundError: No module named 'numpy.strings'

In [ ]:
import cv2
import dlib
import numpy as np
import argparse
import os
import sys
import joblib
from scipy.spatial import distance as dist

# Attempt to import TensorFlow/Keras
try:
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    # REMOVED: model_to_yaml causes import errors in some Keras versions.
    # We will use the more stable built-in 'trained_model.to_json()' method instead,
    # but save the output to a .yaml file as requested by the user.
    from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Rescaling
    from tensorflow.keras.optimizers import Adam
    TENSORFLOW_AVAILABLE = True
except ImportError:
    print("Error: TensorFlow/Keras not found. Please run '!pip install tensorflow' first.")
    TENSORFLOW_AVAILABLE = False
    sys.exit(1)


# --- Configuration (Set Your Paths Here) ---
# NOTE: ⚠️ Path updated based on your input
DEFAULT_VIDEO_PATH = "/content/drive/MyDrive/Video/Screen-Recording (5).mp4"
DLIB_PREDICTOR_PATH = "/content/drive/MyDrive/data_set/shape_predictor_68_face_landmarks.dat"

# Training Output Configuration
DATA_DIR = "cnn_dataset"  # Output folder for generated images (0/ and 1/)
LFB_MODEL_SAVE_PATH = "lfb_fatigue_detector_model.keras"
# REVERTED TO YAML PATH, will contain JSON content for compatibility
LFB_MODEL_YAML_PATH = "lfb_model_architecture.yaml"
IMAGE_SIZE = (48, 48)     # Target size for CNN input
BATCH_SIZE = 32
EPOCHS = 20

# Feature Calculation Constants
EAR_THRESHOLD = 0.23  # More sensitive threshold for fatigue (0.25 is too high for some videos)
ALERT_LABEL = 0
FATIGUE_LABEL = 1
FACE_LANDMARKS = list(range(68))
EYE_CROP_INDICES = list(range(36, 48))
FRAME_SKIP_RATE = 5   # Process only 1 in 5 frames


# ----------------------------------------------
# --- 1. DATA GENERATION FUNCTIONS (Video -> Images) ---
# ----------------------------------------------

def eye_aspect_ratio(eye):
    """Calculates the Eye Aspect Ratio (EAR)."""
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

def get_ear_value(shape):
    """Calculates the average EAR from dlib shape object."""
    landmarks = np.array([(shape.part(i).x, shape.part(i).y) for i in FACE_LANDMARKS])
    left_eye = landmarks[list(range(42, 48))]
    right_eye = landmarks[list(range(36, 42))]
    ear_left = eye_aspect_ratio(left_eye)
    ear_right = eye_aspect_ratio(right_eye)
    return (ear_left + ear_right) / 2.0, landmarks

def generate_cnn_dataset(video_path, predictor_path, output_dir):
    """Processes video frames to create a labeled image dataset (0/ and 1/)."""
    print(f"\n--- 1.1. Starting Image Preparation from: {video_path} ---")

    if not os.path.exists(predictor_path):
        print(f"Error: Dlib predictor not found at {predictor_path}. Cannot generate dataset.")
        return False
    if not os.path.exists(video_path):
        print(f"Error: Video file not found at {video_path}. Cannot generate dataset.")
        return False

    os.makedirs(os.path.join(output_dir, str(ALERT_LABEL)), exist_ok=True)
    os.makedirs(os.path.join(output_dir, str(FATIGUE_LABEL)), exist_ok=True)

    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(predictor_path)
    cap = cv2.VideoCapture(video_path)

    frame_count, alert_count, fatigue_count = 0, 0, 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # Only process a fraction of frames to speed up training and reduce correlation
        if frame_count % FRAME_SKIP_RATE != 0:
            continue

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray, 0)

        if len(faces) > 0:
            rect = faces[0]
            shape = predictor(gray, rect)
            ear_avg, landmarks = get_ear_value(shape)

            # Determine label: EAR below threshold is FATIGUE (1)
            current_label = FATIGUE_LABEL if ear_avg < EAR_THRESHOLD else ALERT_LABEL

            # --- Cropping Logic around the eyes ---
            points_to_crop = landmarks[EYE_CROP_INDICES]
            x_min = np.min(points_to_crop[:, 0]) - 15
            x_max = np.max(points_to_crop[:, 0]) + 15
            y_min = np.min(points_to_crop[:, 1]) - 15
            y_max = np.max(points_to_crop[:, 1]) + 15

            # Clamp coordinates to frame bounds
            x_min = max(0, x_min); y_min = max(0, y_min)
            x_max = min(frame.shape[1], x_max); y_max = min(frame.shape[0], y_max)

            cropped_image = frame[y_min:y_max, x_min:x_max]

            if cropped_image.size != 0:
                resized_image = cv2.resize(cropped_image, IMAGE_SIZE)
                final_image = resized_image # Use BGR image for CNN input (3 channels)

                # Save the image
                label_dir = os.path.join(output_dir, str(current_label))
                image_filename = f"frame_{frame_count:06d}_{ear_avg:.3f}.jpg"
                save_path = os.path.join(label_dir, image_filename)
                cv2.imwrite(save_path, final_image)

                if current_label == ALERT_LABEL:
                    alert_count += 1
                else:
                    fatigue_count += 1

        if frame_count % 500 == 0:
            print(f"Processed {frame_count} frames. Alert saved: {alert_count}, Fatigue saved: {fatigue_count}")

    cap.release()
    print("\n---------------------------------------------------------")
    print(f"✅ Image dataset preparation finished.")
    print(f"Total images saved: Alert={alert_count}, Fatigue={fatigue_count}")
    print("---------------------------------------------------------")

    # Check if we actually generated any fatigue images
    if fatigue_count == 0 and alert_count == 0:
        print("Error: No images were generated. Check video path and dlib predictor.")
        return False
    elif fatigue_count == 0:
        print("Warning: Only ALERT (0) images were generated. CNN training results will be poor.")
        print("Try using a video with more eye closure/blinks, or lower the EAR_THRESHOLD.")

    return True


# ----------------------------------------------
# --- 2. MODEL TRAINING FUNCTIONS (Images -> Model) ---
# ----------------------------------------------

def load_data(data_directory, img_size, batch_size):
    """Loads and prepares the image dataset from the directory structure."""
    print("\n--- 2.1. Loading Data from Disk ---")

    dataset = tf.keras.utils.image_dataset_from_directory(
        data_directory,
        labels='inferred',
        label_mode='binary', # Output is 0 or 1
        image_size=img_size,
        interpolation='bilinear',
        batch_size=batch_size,
        shuffle=True,
        seed=42
    )

    # Normalize pixel values from [0, 255] to [0, 1]
    normalization_layer = Rescaling(1./255)
    dataset = dataset.map(lambda x, y: (normalization_layer(x), y))

    dataset_size = tf.data.experimental.cardinality(dataset).numpy()
    if dataset_size == 0:
        print(f"Error: No batches found in '{data_directory}'. Cannot train LFB model.")
        sys.exit(1)

    # Split dataset: 80% train, 20% validation
    train_size = int(0.8 * dataset_size)
    train_ds = dataset.take(train_size)
    val_ds = dataset.skip(train_size)

    print(f"Total batches found: {dataset_size}. Split: Train={train_size}, Validation={dataset_size - train_size}")

    train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
    val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

    return train_ds, val_ds

def create_cnn_model(input_shape):
    """Defines the LFB CNN model architecture."""
    print("--- 2.2. Defining LFB CNN Model Architecture ---")
    model = Sequential([
        # First Block
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        # Second Block
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        # Third Block
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        # Classification layers
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # Binary output
    ])
    return model

def train_lfb_model(train_ds, val_ds):
    """Compiles and trains the defined LFB CNN model."""

    input_shape = IMAGE_SIZE + (3,)
    model = create_cnn_model(input_shape)

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    model.summary()
    print(f"--- 2.3. Starting LFB Model Training for {EPOCHS} epochs ---")

    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS
    )

    return model

# ----------------------------------------------
# --- 3. MAIN EXECUTION ---
# ----------------------------------------------

if __name__ == "__main__":

    # FIX: Remove the Jupyter/Colab kernel arguments (-f <filename>)
    # that cause the 'unrecognized arguments' error in argparse.
    if '-f' in sys.argv:
        f_index = sys.argv.index('-f')
        # Remove the '-f' flag and the following argument (the json file path)
        sys.argv = sys.argv[:f_index] + sys.argv[f_index+2:]

    # ⚠️ Set up the arguments and path. This allows running it from command line,
    # but defaults to the hardcoded path above.
    parser = argparse.ArgumentParser(description="Full LFB Drowsiness Detection Training Pipeline.")
    parser.add_argument("--video", type=str, default=DEFAULT_VIDEO_PATH,
                        help="Path to the input video file.")
    parser.add_argument("--predictor", type=str, default=DLIB_PREDICTOR_PATH,
                        help="Path to the dlib shape predictor file.")

    args = parser.parse_args()

    # --- Step 1: Data Generation ---
    success = generate_cnn_dataset(args.video, args.predictor, DATA_DIR)

    if not success:
        print("\nPipeline failed at the data generation step. Please check the video and predictor paths.")
        sys.exit(1)

    # --- Step 2: Model Training ---
    try:
        # Load and prepare data
        train_data, val_data = load_data(DATA_DIR, IMAGE_SIZE, BATCH_SIZE)

        # Train the model
        trained_model = train_lfb_model(train_data, val_data)

        # --- Save the model (FULL MODEL) ---
        trained_model.save(LFB_MODEL_SAVE_PATH)

        # --- Save the architecture as YAML (via JSON FIX) ---
        # The .to_json() method is generally available on the Sequential model object.
        # We save this JSON output to a .yaml file to satisfy the user's request.
        json_model_string = trained_model.to_json()
        with open(LFB_MODEL_YAML_PATH, "w") as yaml_file:
            # Note: This file will contain JSON content, but use the .yaml extension.
            yaml_file.write(json_model_string)

        print("\n---------------------------------------------------------")
        print(f"✅ FINAL SUCCESS: LFB Model (Full) saved to: {os.path.abspath(LFB_MODEL_SAVE_PATH)}")
        print(f"✅ LFB Model (YAML Architecture - contains JSON) saved to: {os.path.abspath(LFB_MODEL_YAML_PATH)}")
        print("---------------------------------------------------------")

    except Exception as e:
        print(f"\nError during model training (Step 2): {e}")
        print("Please verify the 'cnn_dataset' folder structure.")
        sys.exit(1)


ModuleNotFoundError: No module named 'numpy.strings'

In [ ]:
#!pip install tensorflow

In [ ]:
#!pip install tensorflow

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib
import zipfile

# ----- Step 1: Extract ZIPs -----
yawn_zip = r"/content/drive/MyDrive/data_set/archive (4).zip"
eye_zip = r"/content/drive/MyDrive/data_set/archive (1).zip"

extract_yawn = r"C:\Users\User\Downloads\yawn_dataset"
extract_eye = r"C:\Users\User\Downloads\eye_blink_dataset"

if not os.path.exists(extract_yawn):
    with zipfile.ZipFile(yawn_zip, 'r') as zip_ref:
        zip_ref.extractall(extract_yawn)

if not os.path.exists(extract_eye):
    with zipfile.ZipFile(eye_zip, 'r') as zip_ref:
        zip_ref.extractall(extract_eye)

print("✅ Datasets extracted successfully!\n")

# ----- Step 2: Mediapipe setup -----
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)

def aspect_ratio(landmarks, indices):
    p1, p2, p3, p4, p5, p6 = [np.array(landmarks[i]) for i in indices]
    vertical1 = np.linalg.norm(p2 - p6)
    vertical2 = np.linalg.norm(p3 - p5)
    horizontal = np.linalg.norm(p1 - p4)
    return (vertical1 + vertical2) / (2.0 * horizontal)

def extract_features(image, part="eye"):
    h, w = image.shape[:2]
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if not results.multi_face_landmarks:
        return None
    landmarks = [(lm.x * w, lm.y * h) for lm in results.multi_face_landmarks[0].landmark]

    if part == "eye":
        EYE = [33, 160, 158, 133, 153, 144]
        feature = aspect_ratio(landmarks, EYE)
    else:
        MOUTH = [78, 308, 13, 14, 87, 317]
        feature = aspect_ratio(landmarks, MOUTH)
    return [feature]

def load_dataset(path, part="eye"):
    X, y = [], []
    subfolders = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))]
    for label, folder in enumerate(subfolders):
        folder_path = os.path.join(path, folder)
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path)
            if img is None:
                continue
            feature = extract_features(img, part)
            if feature:
                X.append(feature)
                y.append(label)
    return np.array(X), np.array(y)

# ----- Step 3: Train Eye Blink Model -----
print("🔹 Training Eye Blink Model...")
X_eye, y_eye = load_dataset(extract_eye, part="eye")
X_train, X_test, y_train, y_test = train_test_split(X_eye, y_eye, test_size=0.2)
eye_model = SVC(kernel='linear')
eye_model.fit(X_train, y_train)
print("Eye Blink Accuracy:", accuracy_score(y_test, eye_model.predict(X_test)))
joblib.dump(eye_model, "eye_blink_model.pkl")
print("✅ Saved: eye_blink_model.pkl")

# ----- Step 4: Train Yawn Model -----
print("\n🔹 Training Yawn Model...")
X_yawn, y_yawn = load_dataset(extract_yawn, part="mouth")
X_train, X_test, y_train, y_test = train_test_split(X_yawn, y_yawn, test_size=0.2)
yawn_model = SVC(kernel='linear')
yawn_model.fit(X_train, y_train)
print("Yawn Detection Accuracy:", accuracy_score(y_test, yawn_model.predict(X_test)))
joblib.dump(yawn_model, "yawn_model.pkl")
print("✅ Saved: yawn_model.pkl")


NameError: name 'audio_classifier' is not defined

In [ ]:
#!pip install Mediapipe